In [1]:
import pandas as pd
import numpy as np

1\. Perform the following operations on plain `txt` files:

+ create a list of integrer numbers and then save it to a text file named "data_int.txt". Run the `cat` command to print the content of the file.
+ create a matrix of 5x5 floats and then save it to a text file named "data_float.txt". Use the `cat` command to print the content of the file.
+ load the txt file of the previous point and convert it to a csv file by hand.

In [2]:
rango = range(100)
print(rango)
f = './data/data_int.txt'
f1 = './data/data_float.txt'
with open(f, mode = 'w') as outfile:
    for i in range(100):
        outfile.write(str(rango[i]) + '\n')
matrix = np.random.random((5,5))
with open(f1, mode = 'w') as outfile:
    outfile.write(str(matrix))
!cat ./data/data_int.txt
!cat ./data/data_float.txt

read_file = pd.read_csv('./data/data_float.txt')
read_file.to_csv('./data/data_convert.csv')

range(0, 100)
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
[[0.19108712 0.03172836 0.11319474 0.93996153 0.96806943]
 [0.89398037 0.76386454 0.05281613 0.55419883 0.39270807]
 [0.72085554 0.97914805 0.92820301 0.58518455 0.48975594]
 [0.97319451 0.81022676 0.04789205 0.74171669 0.40646429]
 [0.15563698 0.87938287 0.42747029 0.45235447 0.63274878]]

2\. Load the file *user_data.json*, which can be found at:

- https://www.dropbox.com/s/sz5klcdpckc39hd/user_data.json

and filter the data by the "CreditCardType" when it equals to "American Express". Than save the data to a new CSV file.

In [6]:
#Tried to fetch the data without downloading the file but it was not possible
import json
import csv
data = json.load(open('./data/user_data.json'))
output = [x for x in data if x['CreditCardType'] == 'American Express']
file = open('./data/user_data.csv', 'w', newline="")
title = "ID,JobTitle,EmailAddress,FirstNameLastName,CreditCard,CreditCardType".split(",")
writer = csv.DictWriter(file, title, delimiter=',', quotechar='|', quoting=csv.QUOTE_MINIMAL)
writer.writeheader()
writer.writerows(output)
!cat ./data/user_data.csv

3\. Load the file from this url:

- https://www.dropbox.com/s/kgshemfgk22iy79/mushrooms_categorized.csv

with Pandas. 

+ explore and print the DataFrame
+ calculate, using `groupby()`, the average value of each feature, separately for each class
+ save the file in a JSON format.

In [4]:
df = pd.read_csv('./data/mushrooms_categorized.csv')
features = df.keys()
means = []
print(features[0])
cont = 1
for i in range(len(features)-1):
    means.append(df.groupby('class')[features[cont]].mean())
    cont += 1
with open('./data/mushrooms_categorized.json', 'w') as outfile:
    for i in range(len(means)):
        json.dump(means[i].to_json(), outfile)
df

class


,class,cap-shape,cap-surface,cap-color,bruises,odor,gill-attachment,gill-spacing,gill-size,gill-color,...,stalk-surface-below-ring,stalk-color-above-ring,stalk-color-below-ring,veil-type,veil-color,ring-number,ring-type,spore-print-color,population,habitat
0,1,5,2,4,1,6,1,0,1,4,...,2,7,7,0,2,1,4,2,3,5
1,0,5,2,9,1,0,1,0,0,4,...,2,7,7,0,2,1,4,3,2,1
2,0,0,2,8,1,3,1,0,0,5,...,2,7,7,0,2,1,4,3,2,3
3,1,5,3,8,1,6,1,0,1,5,...,2,7,7,0,2,1,4,2,3,5
4,0,5,2,3,0,5,1,1,0,4,...,2,7,7,0,2,1,0,3,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8119,0,3,2,4,0,5,0,0,0,11,...,2,5,5,0,1,1,4,0,1,2
8120,0,5,2,4,0,5,0,0,0,11,...,2,5,5,0,0,1,4,0,4,2
8121,0,2,2,4,0,5,0,0,0,5,...,2,5,5,0,1,1,4,0,1,2
8122,1,3,3,4,0,8,1,0,1,0,...,1,7,7,0,2,1,0,7,4,2


4\. Get the binary file named *credit_card.dat* from this address:

- https://www.dropbox.com/s/8m0syw2tkul3dap/credit_card.dat

and convert the data into the real credit card number, knowing that:
- each line corresponds to a credit card number, which consists of 16 characters (which are numbers in the 0-9 range) divided in 4 blocks, with a whitespace between each block
- each character is written using a 6 bit binary representation (including the whitespace)
- the final 4 bits of each line are a padding used to determine the end of the line, and can be ignored

*Hint*: convert the binary numbers to the decimal representation first, and then use the `chr()` function to convert the latter to a char

In [5]:
#!hexdump ./data/credit_card.dat >> ./data/credit_card.csv

files = pd.read_csv('./data/credit_card.csv')
print(files)

     0000000 3131 3130 3131 3131 3130 3031 3131 3130
0    0000010 3030 3131 3031 3030 3031 3030 3030 3131
1    0000020 3130 3130 3131 3130 3031 3131 3130 3131
2    0000030 3131 3030 3131 3031 3030 3030 3131 3030
3    0000040 3131 3131 3130 3131 3131 3130 3131 3131
4    0000050 3130 3130 3031 3030 3030 3131 3030 3031
..                                               ...
368  0001710 3031 3031 3130 3031 3030 3131 3030 3030
369  0001720 3130 3031 3030 3131 3031 3030 3130 3131
370  0001730 3030 3130 3031 3031 3130 3130 0a30 3031
371  0001740 3031 000a                              
372                                          0001743

[373 rows x 1 columns]


5\. **Optional**: load the remote file:

- https://www.dropbox.com/s/aamg1apjhclecka/regression_generated.csv

with Pandas and create a scatter plot with all possible combinations of the following features:
    
  + features_1
  + features_2
  + features_3